In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.utils import load_config
from datasets.SyntheticData import SyntheticData
import numpy as np
import pandas as pd
from torch_geometric.utils import to_dense_adj
import os
from datasets.RealEpidemics import RealEpidemics

## two-step-SINDy

In [ ]:
from models.baseline.TSS.NumericalDerivatives import NumericalDeriv
from models.baseline.TSS.ElementaryFunctions_Matrix import ElementaryFunctions_Matrix
from models.baseline.TSS.TwoPhaseInference import TwoPhaseInference

## Utils

In [ ]:
def load_data_tss(config, snr_db = -1, real_epid=False):
    if not real_epid:
        dataset = SyntheticData(
            root=config['data_folder'],
            dynamics=config['name'],
            t_span=config['t_span'],
            t_max=config['t_eval_steps'],
            num_samples=config['num_samples'],
            seed=config['seed'],
            n_ics=config['n_iter'],
            input_range=config['input_range'],
            device=config['device'],
            horizon = config['horizon'],
            history = config['history'],
            stride=config.get('stride', 5),
            predict_deriv=config.get("predict_deriv", False),
            snr_db=snr_db,
            **config['integration_kwargs']
        )
    else:
        dataset = RealEpidemics(
            root = './data_real_epid_covid_orig',
            name = 'RealEpid',
            predict_deriv=True,
            scale=False,
        )
    
    raw_data = dataset.raw_data_sampled.cpu().detach().numpy() # shape: (ics, time_steps, n_nodes, 1)
    time = dataset.t_sampled
    
    edge_index = dataset[0].edge_index
    A = to_dense_adj(edge_index)[0].cpu().detach().numpy()
    
    return raw_data, A, time


def get_matrix_tss(raw_data, time, A, Dim=1, selfPolyOrder = 3, act_index=False):
    dt = time[0, 1] - time[0, 0]
    dt = dt.item()
    Nnodes = A.shape[0]
    
    data = []
    num_deriv = []
    Matrix = []
    
    for ic in range(raw_data.shape[0]):
        data_ic = raw_data[ic].squeeze(-1)  # shape: (time_steps, n_nodes)
        num_deriv_ic = NumericalDeriv(
            TimeSeries=data_ic,
            dim=1,
            Nnodes=data_ic.shape[1],
            deltT=dt
        )   # pd DatafRame
        
        data_ic = data_ic[2:-2,:]
        data.append(data_ic)
        num_deriv.append(num_deriv_ic)
        matrix_ic = ElementaryFunctions_Matrix(
            data_ic, 
            Dim, 
            Nnodes, 
            A, 
            selfPolyOrder, 
            coupledPolyOrder = 1, 
            PolynomialIndex = True, 
            TrigonometricIndex = True, 
            ExponentialIndex = True, 
            FractionalIndex = False, 
            ActivationIndex = act_index, 
            RescalingIndex = False, 
            CoupledPolynomialIndex = True,
            CoupledTrigonometricIndex = True, 
            CoupledExponentialIndex = True, 
            CoupledFractionalIndex = False,
            CoupledActivationIndex = act_index, 
            CoupledRescalingIndex = False
        )
        
        Matrix.append(matrix_ic)
        

    data = np.concatenate(data, axis=0)
    num_deriv = pd.concat(num_deriv, ignore_index=True)
    Matrix = pd.concat(Matrix, ignore_index=True)
    Matrix = Matrix.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    
    return Matrix, num_deriv, data


def two_step_sindy(Matrix, num_deriv, Nnodes, out_path, Dim = 1, plotstart = 0.5, plotend = 0.9, Keep = 10, SampleTimes = 20, Batchsize = 1,
                   snr_db = -1):
    Lambda = pd.DataFrame([[0.01, 0.5, 1]])
    os.makedirs(out_path, exist_ok=True)
    
    for dim in range(Dim):
        InferredResults, _, _, _ = TwoPhaseInference(
            Matrix, 
            num_deriv, 
            Nnodes, 
            dim, 
            Dim, 
            Keep, 
            SampleTimes,
            Batchsize, 
            Lambda, 
            plotstart, 
            plotend
        )
        
        save_file = f"{out_path}/results_dim={dim}.csv" if snr_db < 0 else f"{out_path}/results_dim={dim}_{snr_db}_db.csv"
        InferredResults.to_csv(save_file)
    
    
        

## Two Phase Inference

### Clean data

In [ ]:
configs = [
    'configs/config_pred_deriv/config_ic1/config_kuramoto.yml',
    'configs/config_pred_deriv/config_ic1/config_biochemical.yml',
    'configs/config_pred_deriv/config_ic1/config_epidemics.yml',
    'configs/config_pred_deriv/config_ic1/config_population.yml'
]

for conf_path in configs:
    conf= load_config(config_path=conf_path)
    raw_data, A, time = load_data_tss(conf)
    
    Matrix, num_deriv, _ = get_matrix_tss(
        raw_data=raw_data,
        time = time,
        A=A,
        Dim=1,
        selfPolyOrder=3
    )
    
    two_step_sindy(
        Matrix=Matrix,
        num_deriv=num_deriv,
        Nnodes=A.shape[0],
        out_path=f'./saved_models_optuna/tss/{conf['name']}-{conf['n_iter']}_nofrac'
    )

### Noise

In [ ]:
configs = [
    'configs/config_pred_deriv/config_ic1/config_kuramoto.yml',
    'configs/config_pred_deriv/config_ic1/config_biochemical.yml',
    'configs/config_pred_deriv/config_ic1/config_epidemics.yml',
    'configs/config_pred_deriv/config_ic1/config_population.yml'
]

snr_db_levels = [70, 50, 20]

for conf_path in configs:
    for snr_db in snr_db_levels:
        
        conf = load_config(config_path=conf_path)
        raw_data, A, time = load_data_tss(conf, snr_db=snr_db)
        
        Matrix, num_deriv, _ = get_matrix_tss(
            raw_data=raw_data,
            time = time,
            A=A,
            Dim=1,
            selfPolyOrder=3
        )
        
        two_step_sindy(
            Matrix=Matrix,
            num_deriv=num_deriv,
            Nnodes=A.shape[0],
            out_path=f'./saved_models_optuna/tss/{conf['name']}-{conf['n_iter']}_nofrac',
            snr_db=snr_db
        )

## Post Processing

In [ ]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

from post_processing import set_pytorch_seed, get_test_set, get_symb_test_error
from utils.utils import load_config
import torch


set_pytorch_seed(0)

kur_config = load_config("./configs/config_pred_deriv/config_ic1/config_kuramoto.yml")

KUR = get_test_set(
    dynamics=kur_config['name'],
    device='cuda',
    input_range=kur_config['input_range'],
    **kur_config['integration_kwargs']
)

g_symb = lambda x: torch.sin(x[:, 1] - x[:, 0]).unsqueeze(-1)
h_symb = lambda x: 2.0 + 0.5 * x[:, 1].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=KUR,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")

"""### Epidemics"""

epid_config = load_config("./configs/config_pred_deriv/config_ic1/config_epidemics.yml")

EPID = get_test_set(
    dynamics=epid_config['name'],
    device='cuda',
    input_range=epid_config['input_range'],
    **epid_config['integration_kwargs']
)

g_symb = lambda x: 0.5*x[:, 1].unsqueeze(-1) * (1 - x[:, 0].unsqueeze(-1))
h_symb = lambda x: x[:, 1].unsqueeze(1) - 0.5 * x[:, 0].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=EPID,
    message_passing=True,
    include_time=False,
    is_symb=False
)


ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")

"""### Population"""

pop_config = load_config("./configs/config_pred_deriv/config_ic1/config_population.yml")

POP = get_test_set(
    dynamics=pop_config['name'],
    device='cuda',
    input_range=pop_config['input_range'],
    **pop_config['integration_kwargs']
)

g_symb = lambda x: 0.2*torch.pow(x[:, 1].unsqueeze(-1), 3)
h_symb = lambda x: -0.5 * x[:, 0].unsqueeze(-1) + x[:, 1].unsqueeze(1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=POP,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")

"""### Biochemical"""

bio_config = load_config("./configs/config_pred_deriv/config_ic1/config_biochemical.yml")

BIO = get_test_set(
    dynamics=bio_config['name'],
    device='cuda',
    input_range=bio_config['input_range'],
    **bio_config['integration_kwargs']
)

g_symb = lambda x: (-0.5*x[:, 1] * x[:, 0]).unsqueeze(-1)
h_symb = lambda x: (1.0 - 0.5 * x[:, 0]).unsqueeze(-1)  + x[:, 1].unsqueeze(-1)

test_losses = get_symb_test_error(
    g_symb=g_symb,
    h_symb=h_symb,
    test_set=BIO,
    message_passing=True,
    include_time=False,
    is_symb=False
)

ts_mean = np.mean(test_losses)
ts_var = np.var(test_losses)
ts_std = np.std(test_losses)

print(f"Mean Test loss of symbolic formula: {ts_mean}")
print(f"Var Test loss of symbolic formula: {ts_var}")
print(f"Std Test loss of symbolic formula: {ts_std}")


In [ ]:
import sympy as sp
import json


def get_tss_test_error(
    text_sympy_mapping_g,
    text_sympy_mapping_h,
    row_means,
    test_set,
    result_dict,
    suffix = '',
    method = "dopri5"
):
    g_symb = sp.S(0)
    h_symb = sp.S(0)

    
    for symb_g in text_sympy_mapping_g.keys():
        g_symb += row_means[symb_g] * text_sympy_mapping_g[symb_g]
    for symb_h in text_sympy_mapping_h.keys():
        h_symb += row_means[symb_h] * text_sympy_mapping_h[symb_h]


    try:
        test_losses = get_symb_test_error(
            g_symb=g_symb,
            h_symb=h_symb,
            test_set=test_set,
            message_passing=False,
            include_time=False,
            method=method,
            atol=1e-5,
            rtol=1e-5,
            is_symb=True
        )

        ts_mean = np.mean(test_losses)
        ts_var = np.var(test_losses)
        ts_std = np.std(test_losses)

        print(f"Mean Test loss of symbolic formula: {ts_mean}")
        print(f"Var Test loss of symbolic formula: {ts_var}")
        print(f"Std Test loss of symbolic formula: {ts_std}")
        
        result_dict[f'tss_test_mae_{suffix}'] = ts_mean
        result_dict[f'tss_test_var_{suffix}'] = ts_var
        result_dict[f'tss_test_std_{suffix}'] = ts_std
        
    except AssertionError:
        print("Evaluation failed !")
        result_dict[f'error_{suffix}'] = 'Evaluation failed !'

### KUR

In [ ]:
df = pd.read_csv("./saved_models_optuna/tss/Kuramoto-1/results_dim=0.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
results_kur = {}

x_i, x_j = sp.symbols('x_i x_j')

text_sympy_mapping_g = {
    "sinx1jMinusx1i": sp.sin(x_j - x_i)
}
text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=KUR,
    result_dict=results_kur
)

#### 70 DB

In [ ]:
df = pd.read_csv("./saved_models_optuna/tss/Kuramoto-1/results_dim=0_70_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)
row_means

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

text_sympy_mapping_g = {
    "sinx1jMinusx1i": sp.sin(x_j - x_i)
}
text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=KUR,
    result_dict=results_kur,
    suffix='70db'
)

#### 50 DB

In [ ]:
df = pd.read_csv("./saved_models_optuna/tss/Kuramoto-1/results_dim=0_50_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)
row_means

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

text_sympy_mapping_g = {
    "sinx1jMinusx1i": sp.sin(x_j - x_i)
}
text_sympy_mapping_h = {
    "fracx1": 1/ x_i
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=KUR,
    result_dict=results_kur,
    suffix='50db'
)

#### 20 DB

In [ ]:
df = pd.read_csv("./saved_models_optuna/tss/Kuramoto-1/results_dim=0_20_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)
row_means

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

text_sympy_mapping_g = {
    "x1iexpx1j": x_i * sp.exp(x_j),
    "fracx1jMinusx1i": 1/(x_j - x_i),
    "fracx1ix1j": 1/(x_i * x_j),
    "x1ifracx1j": x_i / x_j
}

text_sympy_mapping_h = {}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=KUR,
    result_dict=results_kur,
    suffix='20db',
    method="rk4"
)

In [ ]:
with open("./saved_models_optuna/tss/Kuramoto-1/post_process_res.json", 'w') as file:
        json.dump(results_kur, file, indent=4)

### EPID

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')
results_epid = {}
df = pd.read_csv("./saved_models_optuna/tss/Epidemics-1/results_dim=0.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "expx1jMinusx1i": sp.exp(x_j - x_i)
}

text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=EPID,
    result_dict=results_epid
)

#### 70 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Epidemics-1/results_dim=0_70_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "expx1jMinusx1i": sp.exp(x_j - x_i)
}

text_sympy_mapping_h = {
    "x1x1x1": x_i * x_i * x_i,
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=EPID,
    result_dict=results_epid,
    suffix='70db'
)

#### 50 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Epidemics-1/results_dim=0_50_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "expx1jMinusx1i": sp.exp(x_j - x_i)
}

text_sympy_mapping_h = {
    "x1x1x1": x_i * x_i * x_i
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=EPID,
    result_dict=results_epid,
    suffix='50db'
)

#### 20 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Epidemics-1/results_dim=0_20_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "fracx1ix1j": 1/(x_i * x_j),
    "x1ifracx1j": x_i / x_j,
    "fracx1jMinusx1i": 1/(x_j - x_i)
}

text_sympy_mapping_h = {}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=EPID,
    result_dict=results_epid,
    suffix='20db',
    method="rk4"
)

In [ ]:
with open("./saved_models_optuna/tss/Epidemics-1/post_process_res.json", 'w') as file:
    json.dump(results_epid, file, indent=4)

### BIO

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Biochemical-1/results_dim=0.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
results_bio = {}

text_sympy_mapping_g = {
    "x1ix1j": x_i * x_j
}

text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=BIO,
    result_dict=results_bio
)

#### 70 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Biochemical-1/results_dim=0_70_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "x1ix1j": x_i * x_j
}

text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=BIO,
    result_dict=results_bio,
    suffix='70db'
)

#### 50 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Biochemical-1/results_dim=0_50_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "x1ix1j": x_i * x_j
}

text_sympy_mapping_h = {
    "x1x1": x_i * x_i
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=BIO,
    result_dict=results_bio,
    suffix='50db',
    method="euler"
)

#### 20 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Biochemical-1/results_dim=0_20_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "fracx1ix1j": 1/(x_i * x_j),
    "x1ifracx1j": x_i / x_j,
    "fracx1jMinusx1i": 1/(x_j - x_i)
}

text_sympy_mapping_h = {}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=BIO,
    result_dict=results_bio,
    suffix='20db'
)

In [ ]:
with open("./saved_models_optuna/tss/Biochemical-1/post_process_res.json", 'w') as file:
    json.dump(results_bio, file, indent=4)

### POP

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Population-1/results_dim=0.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
results_pop = {}

text_sympy_mapping_g = {
    "sinx1j": sp.sin(x_j),
    "x1j": x_j
}

text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=POP,
    result_dict=results_pop
)

#### 70 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Population-1/results_dim=0_70_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "x1j": x_j
}

text_sympy_mapping_h = {
    "constant": sp.S(1.0)
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=POP,
    result_dict=results_pop,
    suffix='70db'
)

#### 50 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Population-1/results_dim=0_50_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {
    "sinx1j": sp.sin(x_j),
    "x1j": x_j
}

text_sympy_mapping_h = {}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=POP,
    result_dict=results_pop,
    suffix='50db'
)

#### 20 DB

In [ ]:
x_i, x_j = sp.symbols('x_i x_j')

df = pd.read_csv("./saved_models_optuna/tss/Population-1/results_dim=0_20_db.csv", header=None)
df.set_index(0, inplace=True)
row_means = df.mean(axis=1)

row_means

In [ ]:
text_sympy_mapping_g = {}

text_sympy_mapping_h = {
    "sinx1": sp.sin(x_i),
    "x1x1": x_i * x_i
}

get_tss_test_error(
    text_sympy_mapping_g=text_sympy_mapping_g,
    text_sympy_mapping_h=text_sympy_mapping_h,
    row_means=row_means,
    test_set=POP,
    result_dict=results_pop,
    suffix='20db'
)

In [ ]:
with open("./saved_models_optuna/tss/Population-1/post_process_res.json", 'w') as file:
        json.dump(results_pop, file, indent=4)

## Re-fitting coefficients

In [ ]:
import torch
import sympy as sp
from post_processing import make_callable, get_model, set_pytorch_seed
import numpy as np

set_pytorch_seed(0)

def build_symb_model_tss():
    x_i, x_j = sp.symbols('x_i x_j')    

    g_symb = (1 / (1 + sp.exp(- (x_j - x_i))))
    h_symb = x_i
    
    g_symb = make_callable(g_symb)
    h_symb = make_callable(h_symb)

    symb_model = get_model(
        g = g_symb,
        h = h_symb,
        message_passing=False,
        include_time=False,
        integration_method='rk4'
    )
    symb_model.predict_deriv = True
    return symb_model


def get_dxdt_pred(data, symb_model, device='cpu'):
    self_int = []
    pair_int = []
    for snapshot in data:
        snapshot = snapshot.to(device)
        _ = symb_model(snapshot)
        self_int.append(symb_model.conv.model.h_out)    # h_out shape = g_out shape = (N, 1)
        pair_int.append(symb_model.conv.model.g_out)
    
    self_int = torch.stack(self_int, dim=1)
    pair_int = torch.stack(pair_int, dim=1)
    
    return self_int.cpu().detach().numpy().flatten(), pair_int.cpu().detach().numpy().flatten()


def sum_over_dxdt(self_int, pair_int, n_nodes, T):
    lib_new = []

    for i in range(n_nodes):
        for t in range(T):
            start = i * T
            end = start + t + 1  # Python slice is exclusive at end
            val1 = np.sum(self_int[start:end])
            val2 = np.sum(pair_int[start:end])  # index 35 in MATLAB = 34 in Python
            lib_new.append([val1, val2])

    lib_new = np.array(lib_new)
    return lib_new



In [ ]:
import os
coeffs_path = "./inferred_coeffs/tpsindy"
os.makedirs(coeffs_path, exist_ok=True)

### COVID

In [ ]:
from datasets.RealEpidemics import RealEpidemics


data_real_epid_orig = RealEpidemics(
    root = './data_real_epid_covid_orig',
    name = 'RealEpid',
    predict_deriv=True,
    scale=False,
)

symb_model_tss = build_symb_model_tss()

self_int, pair_int = get_dxdt_pred(
    data=data_real_epid_orig,
    symb_model=symb_model_tss,
    device=data_real_epid_orig.device
)

lib = sum_over_dxdt(
    self_int=self_int,
    pair_int=pair_int,
    n_nodes=data_real_epid_orig[0].x.shape[0],
    T=len(data_real_epid_orig)
)

import pandas as pd
lib_df = pd.DataFrame(lib, columns=['x', 'sigxjminusxi'])
lib_df.to_csv(f'{coeffs_path}/lib_new_covid.csv', index=False)


### H1N1

In [ ]:
data_real_epid_orig_h1n1 = RealEpidemics(
    root = './data_real_epid_h1n1_orig',
    name = 'RealEpid',
    predict_deriv=True,
    scale=False,
    infection_data="./data/RealEpidemics/infected_numbers_H1N1.csv",
    inf_threshold=100
)

symb_model_tss = build_symb_model_tss()

self_int, pair_int = get_dxdt_pred(
    data=data_real_epid_orig_h1n1,
    symb_model=symb_model_tss,
    device=data_real_epid_orig_h1n1.device
)

lib = sum_over_dxdt(
    self_int=self_int,
    pair_int=pair_int,
    n_nodes=data_real_epid_orig_h1n1[0].x.shape[0],
    T=len(data_real_epid_orig_h1n1)
)

lib_df = pd.DataFrame(lib, columns=['x', 'sigxjminusxi'])
lib_df.to_csv(f'{coeffs_path}/lib_new_h1n1.csv', index=False)

### SARS

In [ ]:
from datasets.RealEpidemics import RealEpidemics
import pandas as pd

data_real_epid_orig_sars = RealEpidemics(
    root = './data_real_epid_sars_orig',
    name = 'RealEpid',
    predict_deriv=True,
    scale=False,
    infection_data="./data/RealEpidemics/infected_numbers_sars.csv",
    inf_threshold=100
)

symb_model_tss = build_symb_model_tss()

self_int, pair_int = get_dxdt_pred(
    data=data_real_epid_orig_sars,
    symb_model=symb_model_tss,
    device=data_real_epid_orig_sars.device
)

lib = sum_over_dxdt(
    self_int=self_int,
    pair_int=pair_int,
    n_nodes=data_real_epid_orig_sars[0].x.shape[0],
    T=len(data_real_epid_orig_sars)
)

lib_df = pd.DataFrame(lib, columns=['x', 'sigxjminusxi'])
lib_df.to_csv(f'{coeffs_path}/lib_new_sars.csv', index=False)

### Fair fitting of coefficients

In [ ]:
from sklearn.linear_model import LinearRegression 

righ_sides = [
    f"{coeffs_path}/lib_new_covid.csv",
    f"{coeffs_path}/lib_new_h1n1.csv",
    f"{coeffs_path}/lib_new_sars.csv"
]

lef_sides = [
    "./inferred_coeffs/tpsindy/left_side_components_covid.csv",
    "./inferred_coeffs/tpsindy/left_side_components_H1N1.csv",
    "./inferred_coeffs/tpsindy/left_side_components_Sars.csv"
]

n_nodes = [82, 21, 4]
names = ['covid', 'h1n1', 'sars']


for j, (rs, ls) in enumerate(zip(righ_sides, lef_sides)):
    X_all = pd.read_csv(rs)
    y_all = pd.read_csv(ls)
    N = n_nodes[j]
    X_mat = X_all.values
    y_mat = y_all.values
    num = len(X_mat[0])
    num2 = len(y_mat[0])
    L = int(len(X_mat)/N)
    times = N
    Coef = np.zeros(shape=(2,times))
    for i in range(0,times):
        X = X_all.iloc[i*L:(i+1)*L,:]
        y = y_all.iloc[i*L:(i+1)*L,:]
        
        cutoff = int(0.9 * len(X))
        X = X.iloc[:cutoff, :]
        y = y.iloc[:cutoff, :]
        
        v1 = X['x']
        v2 = X['sigxjminusxi']
        y1 = y['X']
        Xin = pd.concat([v1,v2],axis=1)
        model = LinearRegression(fit_intercept=False)
        model.fit(Xin,y1)
        a = model.coef_
        a = (pd.DataFrame(a)).values
        Coef[0,i] = a[0]
        Coef[1,i] = a[1]
        
    Coef = pd.DataFrame(Coef)
    # print(Coef)
    Coef.to_csv(f"{coeffs_path}/inf_coeffs_test_{names[j]}.csv", index=0)

